In [1]:
from mpl_toolkits.basemap import Basemap
import wradlib as wrl
import numpy as np
import matplotlib.pyplot as pl
%matplotlib
import os
# import matplotlib as mpl
from osgeo import osr
import netCDF4
from numpy import arange, dtype

Using matplotlib backend: TkAgg


In [4]:
rx_filename = 'raa01-rx_10000-1710020645-dwd---bin'
rxdata, rxattrs = wrl.io.read_RADOLAN_composite(rx_filename)
rxdata = np.ma.masked_equal(rxdata, -9999) / 2 - 32.5

In [5]:
# Get coordinates

radolan_grid_ll = wrl.georef.get_radolan_grid(900,900, wgs84=True)

proj_stereo = wrl.georef.create_osr("dwd-radolan")

lon_wgs0 = radolan_grid_ll[:,:,0]
lat_wgs0 = radolan_grid_ll[:,:,1]

datestring=rxattrs['datetime'].strftime("%Y-%m-%d %H:%M")

In [110]:
# Convert to netcdf
ncfile=netCDF4.Dataset('out.nc','w')

calendar = 'standard'
units = 'hours since 1970-01-01 00:00'
times=[rxattrs['datetime']]

ncfile.createDimension('nlon',900)
ncfile.createDimension('nlat',900)
ncfile.createDimension('time',None)

x = ncfile.createVariable('lon2d', 'd', ('nlon','nlat'))
y = ncfile.createVariable('lat2d', 'd', ('nlon','nlat'))

x.units = 'degrees_east'
y.units = 'degrees_north'

timevar = ncfile.createVariable(varname='time', dimensions=('time',) ,datatype='float64')
timevar.units = units
timevar[:] = netCDF4.date2num(times, units, calendar=calendar)
 
# transfer the coordinate variables:
x[:] = lon_wgs0
y[:] = lat_wgs0

# Create data variable in NetCDF.
data = ncfile.createVariable('rxdata', 'd', ('time','nlon','nlat'))
data.units='dbZ'
 
# transfer the data variables:
data[0,:,:] = rxdata

ncfile.close()

In [46]:
# plot RX product
# fig = pl.figure()

bmap = Basemap(projection='cyl', llcrnrlon=4, llcrnrlat=47, urcrnrlon=16, urcrnrlat=55,  resolution='i')

bmap.contourf(lon_wgs0,lat_wgs0,rxdata,np.arange(-33,60,0.5),cmap='nipy_spectral')

# Draw the coastlines, countries, parallels and meridians
bmap.drawcoastlines(linewidth=0.5, linestyle='solid', color='white')
bmap.drawcountries(linewidth=0.5, linestyle='solid', color='white')
bmap.drawparallels(np.arange(-90.0, 90.0, 1), linewidth=0.1, color='white', labels=[True, False, False, True])
bmap.drawmeridians(np.arange(0.0, 360.0, 1), linewidth=0.1, color='white', labels=[True, False, False, True])
bmap.readshapefile('/Users/guidocioni/shapefiles/DEU_adm_shp/DEU_adm1','DEU_adm1',linewidth=0.3,color='white')

# Insert the legend
bmap.colorbar(location='right', label='Reflectivity [dBz]')

pl.title('RX Product single scan\n' + rxattrs['datetime'].isoformat())

# pl.grid(color='r')
pl.show()
# pl.savefig('radar'+ rxattrs['datetime']+'.png', dpi=150)